<a href="https://colab.research.google.com/github/bolongliu/devtoolkits/blob/master/hugggingface_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio bypy huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 

In [2]:
import os
import shutil

from huggingface_hub import snapshot_download

from bypy import ByPy

### 下载模型

In [3]:
def copy_data(source_directory,target_directory):
  # 遍历源目录中的所有文件和目录
  for entry in os.listdir(source_directory):
      entry_path = os.path.join(source_directory, entry)
      # 检查是否为符号链接
      if os.path.islink(entry_path):
          # 获取符号链接的目标路径
          target_path = os.readlink(entry_path)
          # 构建目标路径的绝对路径
          target_absolute_path = os.path.join(source_directory, target_path)
          # 获取目标路径的文件名或目录名
          target_basename = os.path.basename(target_absolute_path)
          # 构建目标文件或目录的路径
          target_entry_path = os.path.join(target_directory, entry)

          # 复制目标文件或目录到目标目录
          if os.path.isdir(target_absolute_path):
              shutil.copytree(target_absolute_path, target_entry_path)
          else:
              shutil.copy2(target_absolute_path, target_entry_path)
      else:
          # 如果不是符号链接，直接复制到目标目录
          shutil.copy2(entry_path, os.path.join(target_directory, entry))

In [4]:
def download(repo_id):
    source_directory = snapshot_download(repo_id=repo_id,cache_dir="/app/data/",force_download =True)
    print("文件路径:",source_directory)
    path_name = repo_id.replace("/","---")
    target_directory = f"{os.getcwd()}/download/{path_name}"
    # 移动文件
    os.system(f"mkdir -p {target_directory}")
    os.system(f"mkdir -p {os.getcwd()}/tar/{path_name}/")
    copy_data(source_directory,target_directory)
    print("复制路径：",target_directory)

    return target_directory

In [5]:
## 设置 repo_id
repo_id = "THUDM/chatglm2-6b-int4"

In [6]:
out_path = download(repo_id)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

文件路径: /app/data/models--THUDM--chatglm2-6b-int4/snapshots/382cc704867dc2b78368576166799ace0f89d9ef
复制路径： /content/download/THUDM---chatglm2-6b-int4


In [7]:
out_path

'/content/download/THUDM---chatglm2-6b-int4'

### 上传百度网盘

百度云开发者平台： https://pan.baidu.com/union

大小限制

所有开发者均可接入使用接口，但可上传单个文件大小根据授权用户的身份有不同的限制:

- 普通用户单个上传文件大小上限为4GB
- 会员用户单个上传文件大小上限为10GB
- 超级会员用户单个上传文件大小上限为20GB

In [ ]:
def upload_by_file(source_path):
    bp = ByPy()
    dest_path = source_path.split("/")[-1]  # 文件名
    out = bp.upload(
      source_path,
      dest_path
    )
    return out
def upload_by_path(source_path,compress=True):
    bp = ByPy()
    # 创建目录
    dest_path = source_path.split("/")[-1]
    # 压缩
    if compress:
      print("分卷压缩....")
      tar_path =f"{os.getcwd()}/tar/{dest_path}"
      os.system(f"zip -s 4000M {tar_path}/{dest_path}.zip {source_path}/*")
      source_path = tar_path
    print("上传百度云....")
    bp.mkdir(dest_path)
    # 同步目录
    out = bp.syncup(
      source_path,
      dest_path
    )
    return out

In [ ]:
bp = ByPy()
print(bp.list())

In [ ]:
# 上传文件夹
# 如果单个文件大小小于 4G 则无需压缩
upload_by_path(out_path,compress=False)

### 删除模型

In [ ]:
!rm -rf /content/download/*
!rm -rf /content/tar/*
